In [28]:
from keras_segmentation.models.segnet import *
import numpy as np
from keras_segmentation.predict import predict_multiple, predict, evaluate
from keras_segmentation.models import all_models
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf

train_images =  "training_set/train_img/"
train_annotations = "training_set/train_ana/"
val_images="training_set/val_img/"
val_annotations="training_set/val_ana/"

In [46]:
model_name = ["vgg_segnet", "resnet50_segnet", "vgg_unet", "resnet50_unet", "vgg_pspnet", "resnet50_pspnet"]
cm = 1
m_version = '_1.0'
model = all_models.model_from_name[model_name[cm]](n_classes=7, input_height=1056, input_width=1440)

if cm % 2 == 0:
    L_num = 15
else:
    L_num = 141

freeze = True
if freeze:
    m_version = m_version+"_frozen"
    for L in range(L_num):
        model.get_layer(index=L).trainable = False
model.summary(show_trainable=True)

print("Using "+model_name[cm]+m_version+" model")


Model: "model_107"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_27 (InputLayer)          [(None, 1056, 1440,  0           []                               N          
                                 3)]                                                                         
                                                                                                             
 zero_padding2d_126 (ZeroPaddin  (None, 1062, 1446,   0          ['input_27[0][0]']               N          
 g2D)                           3)                                                                           
                                                                                                             
 conv1 (Conv2D)                 (None, 528, 720, 64  9472        ['zero_padding2d_126[0][0]']     N  

In [47]:
### Only for tensorboard callback
# log_dir="logs/fit/"+model_name[cm]+m_version
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoints_path = "./checkPoints/"+model_name[cm]+m_version+"/"+model_name[cm]
out_dir="output_dir/"+model_name[cm]+m_version

model.train(
    train_images =  "training_set/train_img/",
    train_annotations = "training_set/train_ana/",
    validate=True,
    val_images="training_set/val_img/",
    val_annotations="training_set/val_ana/",
    auto_resume_checkpoint=True,
    checkpoints_path = checkpoints_path, 
    epochs=1,
    batch_size=4,
    steps_per_epoch=8,
    val_batch_size=2,
    val_steps_per_epoch=8,
    do_augment=True
)

Verifying training dataset


100%|██████████| 185/185 [00:07<00:00, 24.29it/s]


Dataset verified! 
Verifying validation dataset


100%|██████████| 75/75 [00:03<00:00, 24.84it/s]


Dataset verified! 


2022-06-20 15:48:49.042186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - ETA: 0s - loss: 2.1506 - accuracy: 0.3321

2022-06-20 15:49:09.436733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: saving model to ./checkPoints/resnet50_segnet_1.0_frozen/resnet50_segnet.00001
8/8 [==============================] - 26s 3s/step - loss: 2.1506 - accuracy: 0.3321 - val_loss: 27.3585 - val_accuracy: 0.4034


In [50]:
out_dir="output_dir/"+model_name[cm]+m_version
# checkpoints_path = ''

predict_multiple(
        model = model,
        inp_dir = val_images, 
        # checkpoints_path = checkpoints_path, 
        out_dir = out_dir+"/nomalSeg")


predict_multiple(
        model = model, 
        inp_dir=val_images, 
        out_dir=out_dir+"/overlay",
        # checkpoints_path=checkpoints_path, 
        overlay_img=True,
        class_names=['_background_', 'normal', 'spiral', 'loose', 'tensioned', 'twisted', 'damage'], 
        show_legends=True)

100%|██████████| 75/75 [00:24<00:00,  3.03it/s]


[array([[3, 6, 5, ..., 2, 2, 2],
        [5, 5, 5, ..., 2, 2, 2],
        [5, 5, 5, ..., 2, 2, 2],
        ...,
        [5, 1, 1, ..., 1, 1, 2],
        [1, 1, 1, ..., 1, 0, 2],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[3, 6, 5, ..., 2, 2, 2],
        [3, 6, 5, ..., 2, 2, 2],
        [3, 5, 5, ..., 2, 2, 2],
        ...,
        [5, 1, 0, ..., 1, 1, 2],
        [1, 1, 1, ..., 1, 0, 2],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[3, 6, 5, ..., 2, 2, 2],
        [5, 5, 5, ..., 4, 2, 2],
        [5, 5, 5, ..., 2, 2, 2],
        ...,
        [5, 1, 1, ..., 1, 1, 2],
        [1, 1, 1, ..., 1, 0, 2],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[3, 5, 2, ..., 2, 2, 2],
        [3, 5, 5, ..., 2, 2, 2],
        [3, 5, 5, ..., 2, 2, 2],
        ...,
        [5, 0, 0, ..., 1, 0, 2],
        [1, 1, 0, ..., 1, 0, 2],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[3, 5, 3, ..., 2, 2, 2],
        [3, 5, 5, ..., 2, 2, 2],
        [3, 5, 5, ..., 2, 2, 2],
        ...,
        [3, 1, 1, ..., 1, 0, 2],
   

In [55]:
ev = model.evaluate_segmentation(inp_images_dir=val_images, annotations_dir=val_annotations)
assert ev['frequency_weighted_IU'] > 0.01
print(ev)

more_lines = ['','','', "Model Name:", str(model_name[cm]+m_version),"checkpoints_path: ", checkpoints_path, "model IOU: ", str(ev)]
with open('models_evaluate.txt', 'a') as f:
    f.writelines('\n'.join(more_lines))

75it [00:25,  2.93it/s]

{'frequency_weighted_IU': 0.35184045464068736, 'mean_IU': 0.058036877300189235, 'class_wise_IU': array([3.61791218e-01, 4.43394308e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.27492297e-04, 0.00000000e+00])}


In [34]:
# checkpoints_path = ''

out = predict(inp="test.png", checkpoints_path = checkpoints_path,
                overlay_img= True, 
                out_fname='test_out.png',
                class_names=['_background_', 'normal', 'spiral', 'loose', 'tensioned', 'twisted', 'damage'], 
                show_legends=True)

loaded weights  ./checkPoints/vgg_segnet_1.0/vgg_segnet.00001


2022-06-20 15:23:14.011480: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [48]:
# %tensorboard --logdir checkpoints/vgg